In [1]:
import numpy as np
import tensorflow as tf
import math

def rosenbrock(x, y, a, b):
    return (a - x)**2 + b*(y - x**2)**2

/home/andrey/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# эксперименты будут случайными
def get_random_rosenbrock_data_points(m):
    result = np.zeros((m, 3))
    result[:, 0] = np.random.uniform(-2, 2, m)
    result[:, 1] = np.random.uniform(-2, 2, m)
    result[:, 2] = rosenbrock(result[:, 0], result[:, 1], 2.5, 2.5)
    return result

m = 500
data_points = get_random_rosenbrock_data_points(m)
# overfitting - не тема этой статьи, но совсем без валидации никак
validation_data_points = get_random_rosenbrock_data_points(m)

In [3]:
# хотим скрытый слой из 10 "нейронов"
n_hidden = 10

# начальную догадку будем конструировать алгоритмом Xavier'a
initializer = tf.contrib.layers.xavier_initializer()

# тензоры весов и смещения для вычисления скрытого уровня
W1 = tf.Variable(initializer([2, n_hidden], dtype=tf.float64))
b1 = tf.Variable(initializer([1, n_hidden], dtype=tf.float64))
# тензоры весов и смещения для вычисления прогноза
W2 = tf.Variable(initializer([n_hidden, 1], dtype=tf.float64))
b2 = tf.Variable(initializer([1], dtype=tf.float64))

with tf.name_scope("net"):
    # тензоры данных экспериментов
    x = tf.placeholder(tf.float64, shape=[m, 2])
    y = tf.placeholder(tf.float64, shape=[m, 1])

    # вычисление скрытого слоя, используем tanh для активации
    h1 = tf.nn.tanh(tf.matmul(x, W1) + b1)
    
    # вычисление прогноза
    y_hat = tf.matmul(h1, W2) + b2
    # невязки
    r = y - y_hat

# данные для подстановки в тензоры placeholder
feed_dict = {x: data_points[:,0:2], 
             y: data_points[:,2:3]}

In [4]:
def jacobian(y, x):
    with tf.name_scope("Jacobian"):
        loop_vars = [
            tf.constant(0, tf.int32),
            tf.TensorArray(tf.float64, size=m),
        ]

        _, jacobian = tf.while_loop(
            lambda i, _: i < m,
            lambda i, res: (i+1, res.write(i, tf.reshape(tf.gradients(y[i], x), (-1,)))),
            loop_vars)
        return jacobian.stack()

r_flat = tf.squeeze(r)
parms = [W1, b1, W2, b2]
j = tf.concat([jacobian(r_flat, p) for p in parms], 1)
jT = tf.transpose(j)

In [5]:
session = tf.Session()
session.run(tf.global_variables_initializer())
session.run(j, feed_dict)
tf.summary.FileWriter('./logs/loss', session.graph)